In [10]:
# import libraries we'll use
import spacy # fast NLP
import pandas as pd # dataframes
import langid # language identification (i.e. what language is this?)
from nltk.classify.textcat import TextCat # language identification from NLTK
from matplotlib.pyplot import plot # not as good as ggplot in R :p
import regex as re

In [11]:
AirData=pd.read_csv("reviews.csv")

In [12]:
AirData["comments"]=AirData["comments"].astype("str")

In [13]:
ids_langid = AirData["comments"].apply(langid.classify)

# get just the language label
langs = ids_langid.apply(lambda tuple: tuple[0])

# how many unique language labels were applied?
print("Number of tagged languages (estimated):")
print(len(langs.unique()))

# percent of the total dataset in English
print("Percent of data in English (estimated):")
print((sum(langs=="en")/len(langs))*100)

Number of tagged languages (estimated):
49
Percent of data in English (estimated):
95.1476374041251


In [45]:
English_comment=AirData["comments"][langs=='en'].str.replace('[^\w\s]','')


In [46]:
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

In [47]:
import io

def clean_html(raw_html): #Cleaning the reviews are required else Carrot2 fails
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    res = re.sub('[^\w\s]', '', raw_html) 
    return cleantext

In [48]:
description=English_comment
description.str.encode('utf-8')#Your reviews might be here as a Series
reviews_descriptions=[]

In [49]:
temp_list=description.tolist() #Convert your reviews in a list
for item in temp_list:
    if item in punc:
        temp_list.remove(item)
for item in temp_list: #Walk through each review
    clean_string=clean_html(item) #Clean it
    clean_string=clean_string.replace('\n','').replace('/','')
    reviews_descriptions.append(clean_string) #Append cleaned review to a list and then use this list to generate xml for Carrot2
print(reviews_descriptions[0])

Our stay with Marcus in Bristol was fantastic in every way He was a great host  picking us up at the bus stop recommending places to try leaving plenty of pastries and other breakfast items to enjoy in the morning The flat itself was modern bright clean and spacious  and best of all right on Bristols lovely harbourside We will definitely stay again next time were in Bristol  thanks again Marcus


In [50]:
def write_to_file(xml):
    with io.open('airbnb_reviews_1000.xml', "w", encoding="utf-8") as f:
        f.writelines(xml)

In [52]:
#### Code for generating XML, instead of printing, you can output contents to a file, i.e. some_file.xml
def generate_xml_file(reviews_description):
    xml = ['<?xml version="1.0" encoding="UTF-8"?>\n']
    xml.append('<searchresult>\n')
    i=0
    for item in reviews_description[1000:2000]: #this is the same list we created above
        xml.append('<document id="{0}">\n'.format(i)) #we can create a document id auto generate that
        xml.append('<title>\n')
        xml.append('{0}\n'.format(item))
        xml.append('</title>\n')
        xml.append('</document>\n')
        i+=1
    xml.append('</searchresult>')
    write_to_file(xml)
    
generate_xml_file(reviews_descriptions)